# Fine-tuning on pair classification task

## Imports

In [1]:
!pip3 install gdown stanza spacy nltk torch==1.13.1 accelerate transformers evaluate --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.5/802.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 75.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 kB 29.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.1 MB/s eta 

In [2]:
from google.colab import drive

from ast import literal_eval
from collections import Counter, defaultdict
import locale
import logging
import re
import random
import os
import pandas as pd
import pickle
import numpy as np
import sklearn
from sklearn.metrics import classification_report
import stanza
import spacy
import torch

import transformers
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from tqdm import tqdm

import evaluate

stanza.download('ru')

INFO:stanza:Downloading default packages for language: ru (Russian) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [5]:
!gdown https://drive.google.com/drive/folders/1bloHcXb5tiBaHjNuX6ZTVPBrm4lXWsNN -O automobiles --folder
!gdown https://drive.google.com/drive/folders/1HEqeIrbSkopfQd0JrHLxjq_jCS31tkaf -O restaurants --folder
!gdown https://drive.google.com/drive/folders/1iXVQwGjn1YKeFAr1NgBn3P99-ctpP9Y- -O both --folder

Retrieving folder list
Retrieving folder 1--gufzIJ_sFCQAG41l6gwPCQl3PXwjsv .ipynb_checkpoints
Processing file 1-PyXsFOOJiTFv_u7qi7W9tl4W3LiGJvL eval_bert_dataset.csv
Processing file 1Y2E6471hlLBGlh2TKNXRBy6_8V2zt9TG eval_model_dataset.csv
Processing file 1-HuGW0DaY9tRuZCYKW6a9u4w9G4KRTrx eval_xlmroberta_dataset.csv
Processing file 1-Oa6L8yfQhriXTuOFxm835-dJli9vf-5 test_bert_dataset.csv
Processing file 1GmAL_4MfntMBmFqfUbhoCMoP4Zbd0Fq1 test_model_dataset.csv
Processing file 1-FbxwKGr8GV27qmOv5DTgRp0NeweqHHp test_xlmroberta_dataset.csv
Processing file 1-QXsb8atrpV948xH0Zq-IUB0EmM6Baqd train_bert_dataset.csv
Processing file 1aZOiztlKWmMPqhjcBbTel_9gTls-gYZz train_model_dataset.csv
Processing file 1-N9fBOka5zHcnQm0uU_9FvtGKTafP_ZE train_xlmroberta_dataset.csv
Retrieving folder list completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1-PyXsFOOJiTFv_u7qi7W9tl4W3LiGJvL
To: /content/automobiles/eval_bert_dataset.csv


In [6]:
def seed_everything(seed=42) -> None:
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False

In [7]:
seed_everything()

In [8]:
SENTIMENT = ['neutral', 'positive', 'negative']
CATEGORIES = ['Whole', 'Interior', 'Service', 'Food', 'Price', 'Comfort', 'Appearance', 'Reliability', 'Safety', 'Driveability', 'Costs']
CATEGORIES_SENTIMENT = [f'{cat} {sent}' for cat in CATEGORIES for sent in SENTIMENT]

In [9]:
id2label_category = {i: label for i, label in enumerate(CATEGORIES)}
label2id_category = {v: k for k, v in id2label_category.items()}

id2label_sentiment = {i: label for i, label in enumerate(SENTIMENT)}
label2id_sentiment = {v: k for k, v in id2label_sentiment.items()}

id2label_category_sentiment = {i: label for i, label in enumerate(CATEGORIES_SENTIMENT)}
label2id_category_sentiment = {v: k for k, v in id2label_category_sentiment.items()}

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## ABSA as NLI


A free interpretation of [this](https://aclanthology.org/W19-6120.pdf) article. In short: aspect-oriented sentiment analysis can be turned into the task of finding a connection between a premise and a hypothesis. In our case, a sentence containing an aspect is a premise, the aspect itself is a hypothesis, and they have some label (for example, tonality or category). How to do?
* Prepare the data. We need triples of the form premise-hypothesis-category.
* Fine-tune models
* Find aspects in the test data. It is possible again with the help of a thesaurus and the best collocations for some metric.

## Data

In [11]:
class ReviewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [12]:
def get_dataset(path: str, tokenizer: AutoTokenizer) -> tuple:
    '''
    Get dataset from files.
    '''
    dataset = pd.read_csv(path, delimiter='\t')
    data = dataset['text'].values.tolist()
    category_ids = dataset['category'].values.tolist()
    sentiment_ids = dataset['sentiment'].values.tolist()

    category_sentiment_ids = [label2id_category_sentiment[f'{id2label_category[cat]} {id2label_sentiment[sent]}'] for cat, sent in zip(category_ids, sentiment_ids)]

    encodings = tokenizer(data, truncation=True, max_length=100, padding=True, add_special_tokens=False)

    category_ds = ReviewsDataset(encodings, category_ids)
    sentiment_ds = ReviewsDataset(encodings, sentiment_ids)
    category_sentiment_ds = ReviewsDataset(encodings, category_sentiment_ids)

    return category_ds, sentiment_ds, category_sentiment_ds

In [13]:
train_bert = '/content/both/train_bert_dataset.csv'
eval_bert = '/content/both/eval_bert_dataset.csv'
test_bert = '/content/both/test_bert_dataset.csv'

train_xlmroberta = '/content/both/train_xlmroberta_dataset.csv'
eval_xlmroberta = '/content/both/eval_xlmroberta_dataset.csv'
test_xlmroberta = '/content/both/test_xlmroberta_dataset.csv'

In [14]:
df = pd.read_csv(train_bert, delimiter='\t')

In [15]:
df['text'].values.tolist()[0]

'[CLS] Я несколько раз была в этом заведении , о кухне могу сказать только самое хорошее . [SEP] заведении [SEP]'

## Model

In [16]:
class NLITrainPipeline:

    def __init__(self, train_ds, val_ds, model_checkpoint, new_model_checkpoint, labels):
        self.train_ds = train_ds
        self.val_ds = val_ds

        self.model_checkpoint = model_checkpoint
        self.new_model_checkpoint = new_model_checkpoint

        self.num_labels = len(labels)
        self.label2id = {label: i for i, label in enumerate(labels)}
        self.id2label = {i: label for i, label in enumerate(labels)}

        self.model = AutoModelForSequenceClassification.from_pretrained(
            self.model_checkpoint,
            num_labels=self.num_labels,
            id2label=self.id2label,
            label2id=self.label2id
            )
        self.tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

        self.training_args = TrainingArguments(
            output_dir='./results',
            num_train_epochs=3,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=64,
            warmup_steps=500,
            weight_decay=0.01,
            logging_dir='./logs',
            logging_steps=100,
        )

        self.clf_report = None

    def train(self):
        '''
        Fine-tune model on the downstream task.
        '''
        self.model.to(device)
        trainer = Trainer(
            model=self.model,
            args=self.training_args,
            train_dataset=self.train_ds,
            eval_dataset=self.val_ds
        )

        trainer.train()
        trainer.save_model(self.new_model_checkpoint)

    def inference(self, path, label):
        '''
        Evaluate model.
        '''
        new_model = AutoModelForSequenceClassification.from_pretrained(
            self.new_model_checkpoint,
            num_labels=self.num_labels,
            id2label=self.id2label,
            label2id=self.label2id
            )
        new_model.to(device)

        true_labels = []
        predicted_labels = []

        test_ds = pd.read_csv(path, delimiter='\t')

        for idx, row in test_ds.iterrows():
            tokenized = self.tokenizer(
                row['text'], return_tensors='pt',
                truncation=True, max_length=100,
                padding=True, add_special_tokens=False
                )
            tokenized = tokenized.to(device)
            with torch.no_grad():
                classification_logits = new_model(**tokenized).logits
            predicted_class_id = classification_logits.to('cpu').argmax().item()
            true_labels.append(new_model.config.id2label[row[label]])
            predicted_labels.append(new_model.config.id2label[predicted_class_id])

        self.clf_report = classification_report(true_labels, predicted_labels)

        return self.clf_report

In [17]:
class NLITrainPipelineCatSent(NLITrainPipeline):

    def __init__(self, *args, **kwargs):
        super(NLITrainPipelineCatSent, self).__init__(*args, **kwargs)

    def inference(self, path):
        '''
        Evaluate model.
        '''
        new_model = AutoModelForSequenceClassification.from_pretrained(
            self.new_model_checkpoint,
            num_labels=self.num_labels,
            id2label=self.id2label,
            label2id=self.label2id
            )
        new_model.to(device)

        true_labels = []
        predicted_labels = []

        test_ds = pd.read_csv(path, delimiter='\t')

        for idx, row in test_ds.iterrows():
            tokenized = self.tokenizer(
                row['text'], return_tensors='pt',
                truncation=True, max_length=100,
                padding=True, add_special_tokens=False
                )
            tokenized = tokenized.to(device)
            with torch.no_grad():
                classification_logits = new_model(**tokenized).logits
            predicted_class_id = classification_logits.to('cpu').argmax().item()
            true_labels.append(f"{id2label_category[row['category']]} {id2label_sentiment[row['sentiment']]}")
            predicted_labels.append(new_model.config.id2label[predicted_class_id])

        self.clf_report = classification_report(true_labels, predicted_labels, output_dict=True)

        return classification_report(true_labels, predicted_labels)

    def sentiment_inference(self):
        '''
        Inference on sentiment to the category-sentiment model.
        '''
        if not self.clf_report:
            raise ValueError('Classification report is empty!')

        sentiment_report = {
            'neutral': {'precision': 0, 'recall': 0, 'f1-score': 0},
            'positive': {'precision': 0, 'recall': 0, 'f1-score': 0},
            'negative': {'precision': 0, 'recall': 0, 'f1-score': 0}
        }

        for label, metrics in self.clf_report.items():
            if label not in ['accuracy', 'macro avg', 'weighted avg']:
                sent_label = label.split()[1]
                if sent_label in sentiment_report:
                    sent_dict = sentiment_report[sent_label]
                    for metric, score in metrics.items():
                        if metric in sent_dict:
                            sent_dict[metric] += (score / len(CATEGORIES))

        return pd.DataFrame(sentiment_report)

In [18]:
# usage
# tokenized = tokenizer(text, return_tensors="pt")
# classification_logits = model(**tokenized).logits
# results = torch.softmax(classification_logits, dim=1).tolist()[0]
# for i in range(len(CATEGORIES)):
#     print(f"{CATEGORIES[i]}: {int(results[i] * 100)}%")

## ruBERT

In [19]:
model_checkpoint = 'ai-forever/ruBert-base'

In [20]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [21]:
train_category_ds, train_sentiment_ds, train_category_sentiment_ds = get_dataset(train_bert, tokenizer)
eval_category_ds, eval_sentiment_ds, eval_category_sentiment_ds = get_dataset(eval_bert, tokenizer)

In [22]:
train_category_ds[0]

{'input_ids': tensor([  101,   119,  1456,   768,  1073,   113,  1019, 46789,   121,   104,
         14726,  1385,  2312,   966,  3627, 13093,   126,   102, 46789,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 

### Sentiment model

In [23]:
nli_trainer_rubert = NLITrainPipeline(
    train_sentiment_ds, eval_sentiment_ds,
    model_checkpoint, 'rubert-sentiment-nli_both', SENTIMENT
)
nli_trainer_rubert.train()

Some weights of the model checkpoint at ai-forever/ruBert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not ini

Step,Training Loss
100,1.000400
200,0.647400
300,0.533400
400,0.510400
500,0.444600
600,0.468600
700,0.451200
800,0.412100
900,0.407100
1000,0.300300


In [24]:
print(nli_trainer_rubert.inference(test_bert, 'sentiment'))
del nli_trainer_rubert

              precision    recall  f1-score   support

    negative       0.78      0.81      0.80       472
     neutral       0.65      0.64      0.65       445
    positive       0.92      0.92      0.92      1631

    accuracy                           0.85      2548
   macro avg       0.78      0.79      0.79      2548
weighted avg       0.85      0.85      0.85      2548



### Category model

In [25]:
nli_trainer_rubert = NLITrainPipeline(
    train_category_ds, eval_category_ds,
    model_checkpoint, 'rubert-category-nli_both', CATEGORIES
)
nli_trainer_rubert.train()

Some weights of the model checkpoint at ai-forever/ruBert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not ini

Step,Training Loss
100,2.276700
200,1.339200
300,0.729900
400,0.559000
500,0.483000
600,0.482600
700,0.427700
800,0.380500
900,0.347000
1000,0.286100


In [26]:
print(nli_trainer_rubert.inference(test_bert, 'category'))
del nli_trainer_rubert

              precision    recall  f1-score   support

  Appearance       0.79      0.69      0.74       103
     Comfort       0.88      0.87      0.88       349
       Costs       0.90      0.84      0.87       125
Driveability       0.83      0.85      0.84       338
        Food       0.96      0.98      0.97       447
    Interior       0.93      0.94      0.94       228
       Price       0.91      0.89      0.90        36
 Reliability       0.73      0.76      0.74       178
      Safety       0.71      0.59      0.64        29
     Service       0.95      0.95      0.95       328
       Whole       0.88      0.89      0.88       387

    accuracy                           0.89      2548
   macro avg       0.86      0.84      0.85      2548
weighted avg       0.89      0.89      0.89      2548



### Category and sentiment model

In [27]:
nli_trainer_rubert = NLITrainPipelineCatSent(
    train_category_sentiment_ds, eval_category_sentiment_ds,
    model_checkpoint, 'rubert-category-sentiment-nli_both', CATEGORIES_SENTIMENT
)
nli_trainer_rubert.train()

Some weights of the model checkpoint at ai-forever/ruBert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not ini

Step,Training Loss
100,3.368200
200,2.715800
300,1.871900
400,1.402100
500,1.138300
600,1.111800
700,0.983800
800,0.891700
900,0.841300
1000,0.678400


In [28]:
print(nli_trainer_rubert.inference(test_bert))
print(nli_trainer_rubert.sentiment_inference())
del nli_trainer_rubert

                       precision    recall  f1-score   support

  Appearance negative       0.45      0.17      0.24        30
   Appearance neutral       0.32      0.47      0.38        19
  Appearance positive       0.78      0.72      0.75        54
     Comfort negative       0.61      0.79      0.69        76
      Comfort neutral       0.20      0.11      0.14        18
     Comfort positive       0.93      0.87      0.90       255
       Costs negative       0.71      0.66      0.68        41
        Costs neutral       0.90      0.43      0.58        21
       Costs positive       0.71      0.81      0.76        63
Driveability negative       0.60      0.66      0.63        58
 Driveability neutral       0.59      0.51      0.55        76
Driveability positive       0.78      0.84      0.81       204
        Food negative       0.53      0.63      0.58        30
         Food neutral       0.66      0.70      0.68       117
        Food positive       0.87      0.84      0.86  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

## mBERT

In [29]:
model_checkpoint = 'bert-base-multilingual-cased'

In [30]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [31]:
train_category_ds, train_sentiment_ds, train_category_sentiment_ds = get_dataset(train_bert, tokenizer)
eval_category_ds, eval_sentiment_ds, eval_category_sentiment_ds = get_dataset(eval_bert, tokenizer)

### Sentiment model

In [32]:
nli_trainer_mbert = NLITrainPipeline(
    train_sentiment_ds, eval_sentiment_ds,
    model_checkpoint, 'mbert-sentiment-nli_both', SENTIMENT
)
nli_trainer_mbert.train()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

Step,Training Loss
100,0.944900
200,0.840500
300,0.752100
400,0.688000
500,0.662100
600,0.654900
700,0.639400
800,0.577300
900,0.549800
1000,0.502500


In [33]:
print(nli_trainer_mbert.inference(test_bert, 'sentiment'))
del nli_trainer_mbert

              precision    recall  f1-score   support

    negative       0.65      0.65      0.65       472
     neutral       0.57      0.54      0.56       445
    positive       0.86      0.88      0.87      1631

    accuracy                           0.78      2548
   macro avg       0.70      0.69      0.69      2548
weighted avg       0.77      0.78      0.77      2548



### Category model

In [34]:
nli_trainer_mbert = NLITrainPipeline(
    train_category_ds, eval_category_ds,
    model_checkpoint, 'mbert-category-nli_both', CATEGORIES
)
nli_trainer_mbert.train()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

Step,Training Loss
100,2.279600
200,1.570000
300,1.103800
400,0.893500
500,0.790100
600,0.806000
700,0.735100
800,0.699600
900,0.604700
1000,0.516800


In [35]:
print(nli_trainer_mbert.inference(test_bert, 'category'))
del nli_trainer_mbert

              precision    recall  f1-score   support

  Appearance       0.78      0.63      0.70       103
     Comfort       0.81      0.84      0.82       349
       Costs       0.80      0.81      0.80       125
Driveability       0.84      0.80      0.82       338
        Food       0.91      0.97      0.94       447
    Interior       0.90      0.89      0.90       228
       Price       0.94      0.89      0.91        36
 Reliability       0.65      0.71      0.68       178
      Safety       0.71      0.59      0.64        29
     Service       0.92      0.93      0.93       328
       Whole       0.84      0.81      0.82       387

    accuracy                           0.85      2548
   macro avg       0.83      0.81      0.81      2548
weighted avg       0.85      0.85      0.85      2548



### Category and sentiment model

In [36]:
nli_trainer_mbert = NLITrainPipelineCatSent(
    train_category_sentiment_ds, eval_category_sentiment_ds,
    model_checkpoint, 'mbert-category-sentiment-nli_both', CATEGORIES_SENTIMENT
)
nli_trainer_mbert.train()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

Step,Training Loss
100,3.377500
200,2.819500
300,2.329000
400,1.934700
500,1.743700
600,1.686600
700,1.553800
800,1.385500
900,1.356200
1000,1.163000


In [37]:
print(nli_trainer_mbert.inference(test_bert))
print(nli_trainer_mbert.sentiment_inference())
del nli_trainer_mbert

                       precision    recall  f1-score   support

  Appearance negative       0.00      0.00      0.00        30
   Appearance neutral       0.33      0.37      0.35        19
  Appearance positive       0.52      0.70      0.60        54
     Comfort negative       0.47      0.75      0.58        76
      Comfort neutral       0.20      0.11      0.14        18
     Comfort positive       0.84      0.82      0.83       255
       Costs negative       0.35      0.17      0.23        41
        Costs neutral       0.00      0.00      0.00        21
       Costs positive       0.46      0.76      0.57        63
Driveability negative       0.31      0.36      0.33        58
 Driveability neutral       0.62      0.47      0.54        76
Driveability positive       0.73      0.72      0.72       204
        Food negative       0.41      0.57      0.48        30
         Food neutral       0.63      0.61      0.62       117
        Food positive       0.82      0.82      0.82  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

## XLM-RoBERTa

In [38]:
model_checkpoint = 'xlm-roberta-base'

In [39]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [40]:
train_category_ds, train_sentiment_ds, train_category_sentiment_ds = get_dataset(train_xlmroberta, tokenizer)
eval_category_ds, eval_sentiment_ds, eval_category_sentiment_ds = get_dataset(eval_xlmroberta, tokenizer)

### Sentiment model

In [41]:
nli_trainer_xlmroberta = NLITrainPipeline(
    train_sentiment_ds, eval_sentiment_ds,
    model_checkpoint, 'xlmroberta-sentiment-nli_both', SENTIMENT
)
nli_trainer_xlmroberta.train()

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

Step,Training Loss
100,1.015100
200,0.856600
300,0.718500
400,0.683800
500,0.619700
600,0.679600
700,0.732100
800,0.620200
900,0.590900
1000,0.539100


In [42]:
print(nli_trainer_xlmroberta.inference(test_xlmroberta, 'sentiment'))
del nli_trainer_xlmroberta

              precision    recall  f1-score   support

    negative       0.77      0.79      0.78       472
     neutral       0.65      0.59      0.62       445
    positive       0.90      0.91      0.90      1631

    accuracy                           0.83      2548
   macro avg       0.77      0.76      0.77      2548
weighted avg       0.83      0.83      0.83      2548



### Category model

In [43]:
nli_trainer_xlmroberta = NLITrainPipeline(
    train_category_ds, eval_category_ds,
    model_checkpoint, 'xlmroberta-category-nli_both', CATEGORIES
)
nli_trainer_xlmroberta.train()

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

Step,Training Loss
100,2.285300
200,1.566900
300,1.008300
400,0.798500
500,0.712100
600,0.731500
700,0.617100
800,0.600100
900,0.558700
1000,0.459200


In [44]:
print(nli_trainer_xlmroberta.inference(test_xlmroberta, 'category'))
del nli_trainer_xlmroberta

              precision    recall  f1-score   support

  Appearance       0.78      0.67      0.72       103
     Comfort       0.81      0.89      0.85       349
       Costs       0.83      0.86      0.84       125
Driveability       0.87      0.80      0.84       338
        Food       0.97      0.96      0.97       447
    Interior       0.92      0.91      0.92       228
       Price       0.94      0.81      0.87        36
 Reliability       0.71      0.78      0.74       178
      Safety       0.77      0.59      0.67        29
     Service       0.94      0.95      0.94       328
       Whole       0.88      0.88      0.88       387

    accuracy                           0.88      2548
   macro avg       0.86      0.83      0.84      2548
weighted avg       0.88      0.88      0.88      2548



### Category and sentiment model

In [45]:
nli_trainer_xlmroberta = NLITrainPipelineCatSent(
    train_category_sentiment_ds, eval_category_sentiment_ds,
    model_checkpoint, 'xlmroberta-category-sentiment-nli_both', CATEGORIES_SENTIMENT
)
nli_trainer_xlmroberta.train()

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

Step,Training Loss
100,3.427800
200,2.797400
300,2.249800
400,1.775200
500,1.531200
600,1.480300
700,1.308000
800,1.245300
900,1.156100
1000,1.014700


In [46]:
print(nli_trainer_xlmroberta.inference(test_xlmroberta))
print(nli_trainer_xlmroberta.sentiment_inference())
del nli_trainer_xlmroberta

                       precision    recall  f1-score   support

  Appearance negative       0.50      0.20      0.29        30
   Appearance neutral       0.17      0.16      0.16        19
  Appearance positive       0.67      0.74      0.70        54
     Comfort negative       0.61      0.79      0.69        76
      Comfort neutral       0.00      0.00      0.00        18
     Comfort positive       0.84      0.88      0.86       255
       Costs negative       0.58      0.71      0.64        41
        Costs neutral       0.00      0.00      0.00        21
       Costs positive       0.70      0.79      0.75        63
Driveability negative       0.47      0.60      0.53        58
 Driveability neutral       0.59      0.54      0.56        76
Driveability positive       0.78      0.75      0.76       204
        Food negative       0.47      0.57      0.52        30
         Food neutral       0.60      0.63      0.62       117
        Food positive       0.85      0.84      0.85  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

In [47]:
!mv -if /content/rubert-sentiment-nli_both /content/drive/MyDrive/models
!mv -if /content/mbert-sentiment-nli_both /content/drive/MyDrive/models
!mv -if /content/xlmroberta-sentiment-nli_both /content/drive/MyDrive/models

# Evaluation on laptops

In [48]:
laptop_aspects = pd.read_csv('/content/drive/MyDrive/Summarization/laptop data/aspects-surface.tsv', delimiter='\t')

laptop_aspects.head()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py:211 in wrapper                │
│                                                                                                  │
│   208 │   │   │   │   │   raise TypeError(msg)                                                   │
│   209 │   │   │   │   else:                                                                      │
│   210 │   │   │   │   │   kwargs[new_arg_name] = new_arg_value                                   │
│ ❱ 211 │   │   │   return func(*args, **kwargs)                                                   │
│   212 │   │                                                                                      │
│   213 │   │   return cast(F, wrapper)                                                            │
│   214                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py:331 in wrapper                │
│                                                                                                  │
│   328 │   │   │   │   │   FutureWarning,                                                         │
│   329 │   │   │   │   │   stacklevel=find_stack_level(),                                         │
│   330 │   │   │   │   )                                                                          │
│ ❱ 331 │   │   │   return func(*args, **kwargs)                                                   │
│   332 │   │                                                                                      │
│   333 │   │   # error: "Callable[[VarArg(Any), KwArg(Any)], Any]" has no                         │
│   334 │   │   # attribute "__signature__"                                                        │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/pandas/io/parsers/readers.py:950 in read_csv             │
│                                                                                                  │
│    947 │   )                                                                                     │
│    948 │   kwds.update(kwds_defaults)                                                            │
│    949 │                                                                                         │
│ ❱  950 │   return _read(filepath_or_buffer, kwds)                                                │
│    951                                                                                           │
│    952                                                                                           │
│    953 # iterator=True -> TextFileReader                                                         │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/pandas/io/parsers/readers.py:605 in _read                │
│                                                                                                  │
│    602 │   _validate_names(kwds.get("names", None))                                              │
│    603 │                                                                                         │
│    604 │   # Create the parser.                                                                  │
│ ❱  605 │   parser = TextFileReader(filepath_or_buffer, **kwds)                                   │
│    606 │                                                   

In [ ]:
laptop_reviews = pd.read_csv('/content/drive/MyDrive/Summarization/laptop data/reviews.tsv', delimiter='\t')

laptop_reviews.head()

In [ ]:
LAPTOP_CATEGORIES = list(laptop_aspects.category.unique())
LAPTOP_SENTIMENT = SENTIMENT

LAPTOP_CATEGORIES_SENTIMENT = [f'{cat} {sent}' for cat in LAPTOP_CATEGORIES for sent in LAPTOP_SENTIMENT]

In [ ]:
laptop_id2label_category = {i: label for i, label in enumerate(LAPTOP_CATEGORIES)}
laptop_label2id_category = {v: k for k, v in id2label_category.items()}

laptop_id2label_sentiment = {i: label for i, label in enumerate(LAPTOP_SENTIMENT)}
laptop_label2id_sentiment = {v: k for k, v in id2label_sentiment.items()}

laptop_id2label_category_sentiment = {i: label for i, label in enumerate(LAPTOP_CATEGORIES_SENTIMENT)}
laptop_label2id_category_sentiment = {v: k for k, v in id2label_category_sentiment.items()}

In [ ]:
nlp = stanza.Pipeline('ru', processors='tokenize')

In [ ]:
def get_laptop_data(reviews: pd.DataFrame, aspects: pd.DataFrame) -> tuple:

    review_ids = []
    data = []
    category_ids = []
    sentiment_ids = []

    reviews_sentences = defaultdict(list)

    logging.warning('Start getting data...')

    reviews = reviews[reviews['id'].isin(aspects['text_id'].unique())]

    for rev_idx, rev in reviews.iterrows():
        text_id = rev['id']
        product_id = rev['product_id']
        text = rev['text']

        logging.warning('Text ID: %s' % text_id)

        # stanza processing to parse sentences and start and end characters
        doc = nlp(text)
        logging.warning('Processed by stanza')
        sents_with_end = {}
        for sent in doc.sentences:
            sentence = []
            for token in sent.tokens:
                sentence.append(token.text)
            
            end_idx = token.end_char  # last token in the current sentence
            
            sentence = ' '.join(sentence)
            sents_with_end[end_idx] = sentence
            reviews_sentences['text_id'].append(sentence)

        logging.warning('Got sentences and their ends')

        # catch needed sentence
        rev_aspects = aspects[aspects['text_id'] == text_id]
        logging.warning('Got aspects for the current review')

        for asp_idx, asp in rev_aspects.iterrows():
            mention_end_char = int(asp['term_to'])

            for end_char in sents_with_end:
                if end_char >= mention_end_char:
                    sentence = sents_with_end[end_char]
                    mention = asp['term']
                    data.append(f'[CLS] {sentence} [SEP] {mention} [SEP]')

                    category_ids.append(LAPTOP_CATEGORIES.index(asp['category']))
                    sentiment_ids.append(SENTIMENT.index(asp['sentiment']))

                    review_ids.append(text_id)

                    break  # just one sentence

        logging.warning('Got data for the current review')

    return review_ids, data, category_ids, sentiment_ids, reviews_sentences

In [ ]:
laptop_review_ids, laptop_data, laptop_category_ids, laptop_sentiment_ids, laptop_reviews_sentences = get_laptop_data(laptop_reviews, laptop_aspects)

In [ ]:
new_sent_model = AutoModelForSequenceClassification.from_pretrained(
    '/content/drive/MyDrive/models/rubert-sentiment-nli_both',
    num_labels=3,
    id2label=id2label_sentiment,
    label2id=label2id_sentiment)
new_sent_tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/ruBert-base')

In [ ]:
new_sent_model.to(device)

In [ ]:
true_sentiment = []
predicted_sentiment = []
for text, sent_id in zip(laptop_data, laptop_sentiment_ids):
    tokenized = new_sent_tokenizer(text, return_tensors='pt', truncation=True, max_length=100, padding=True, add_special_tokens=False)
    tokenized = tokenized.to(device)
    with torch.no_grad():
        classification_logits = new_sent_model(**tokenized).logits
    predicted_class_id = classification_logits.to('cpu').argmax().item()
    # label = new_cat_model.config.id2label[predicted_class_id]
    true_sentiment.append(new_sent_model.config.id2label[sent_id])
    predicted_sentiment.append(new_sent_model.config.id2label[predicted_class_id])

In [ ]:
print(classification_report(true_sentiment, predicted_sentiment))